In [1]:
#!/usr/bin/env python
import os
import numpy as np
import sys
import xarray as xr
import pygrib
import matplotlib
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [2]:
# NetCDF4 files

# Set pathway and filename
path_mpi  = '/f2/haboob/home/shared/Haboob_SERDP/CMIP5_MPI_Hires/'
model_data = xr.open_dataset(path_mpi+'wrfout_d03_2010-07-10_06:00:00')
#model_data  # uncomment this to see full list of variables/dimensions of xarray

# Assign one variable in data file RAINNC to script variable 
# select last index -1 in time dimension because RAINNC is hourly accumulated 
pr_model = model_data.RAINNC[-1,:,:]
pr_model.shape

(400, 600)

In [3]:
# GRIB2 files

# Set pathway and filename
path_stageiv = '/net/fractus/f1/luong/SERDP/Stage_IV/raw/'
file = 'ST4.2010071012.24h'
#path_stageiv+file #uncomment to check pathway

In [4]:
# Use pygrib to open grib file
grbs = pygrib.open(path_stageiv+file)
#grbs.read() # uncomment to read all grib information like variables/dimensions 

In [5]:
# Select variables, assign to script variables, and any processing
msg = grbs.select(name='Total Precipitation')[0]  
# Replace the missing values with a more robust missing value, np.nan
precip_obs = np.where(msg.values.data==9999.0, np.nan, msg.values.data)
lats_obs, lons_obs = msg.latlons()
# Check local dimensions
precip_obs.shape,lats_obs.shape,lons_obs.shape

((881, 1121), (881, 1121), (881, 1121))

In [6]:
# Create xarray for obs data for efficient lat/lon subsetting
pr_obs = xr.DataArray(data=precip_obs,
                  dims=["x", "y"],
                  coords=dict(lon=(["x", "y"], lons_obs),
                              lat=(["x", "y"], lats_obs)),
                  attrs=dict(description="Precipitation Rate",
                             units="mm/day")                      )
pr_obs.shape

(881, 1121)

In [7]:
# Crop both files to Arizona co-ordinates
# Define AZ co-ordinates
az_cood = [-115, -109, 31.0, 37.5] 
# Crop model data
az_pr_model = pr_model.where((az_cood[0] <= pr_model.XLONG) & (pr_model.XLONG < az_cood[1])
                      & (az_cood[2] <= pr_model.XLAT) & (pr_model.XLAT < az_cood[3]), drop=True)
# Crop observations
az_pr_obs = pr_obs.where((az_cood[0] <= pr_obs.lon) & (pr_obs.lon < az_cood[1])
                      & (az_cood[2] <= pr_obs.lat) & (pr_obs.lat < az_cood[3]), drop=True)

az_pr_model.shape,az_pr_obs.shape

((290, 228), (196, 159))

In [ ]:
# Panel plot of model vs obs

# set the projection for the map
projection = ccrs.PlateCarree()

# start a figure named fig, set axis name, dimesions of fig, and projection
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(16, 8),
                       subplot_kw={'projection': projection})

## AX1
# set levels for colorbar
levels = np.arange(5,35,5)
# plot filled contours (contourf) onto ax1
cs = ax1.contourf(az_pr_model.XLONG.data, az_pr_model.XLAT.data, az_pr_model.data,
                  transform=projection,
                  cmap='gist_ncar_r',levels=levels,
                  extend='both')

# add color bar
labels = [str(item) for item in levels]
cax,kw = matplotlib.colorbar.make_axes(ax1,ticks=levels,location='right',pad=0.03,shrink=0.7)
cbar = fig.colorbar(cs,cax=cax,**kw)
cbar.ax.set_yticklabels(labels)
cbar.set_label('Precipitation $[mm/day]$',size=10)

# crop extent of map to lat/lon of data
ax1.set_extent(az_cood)

# add lat lon gridline
gl = ax1.gridlines(draw_labels=True,
             xlocs=np.arange(-180, 180, 1.),
             ylocs=np.arange(-90, 90, 1.),
             linewidth=0.5, color='k', alpha=0.5, linestyle=':')
gl.xlabels_top = None
gl.ylabels_right = None
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# set plot title
ax1.set_title('2010-07-10 :: WRF-MPI Model', loc='center',pad=5,fontsize=12) 

# add map features to plot
#ax1.add_feature(cartopy.feature.LAND)
#ax1.add_feature(cartopy.feature.OCEAN)
ax1.add_feature(cartopy.feature.COASTLINE)
ax1.add_feature(cartopy.feature.STATES,linewidth=0.5)
ax1.add_feature(cartopy.feature.BORDERS, linestyle=':',linewidth=0.5)

## AX2
# plot filled contours (contourf) onto ax2
cs = ax2.contourf(az_pr_obs.lon, az_pr_obs.lat, az_pr_obs,
                  transform=projection,
                  cmap='gist_ncar_r',levels=levels,
                  extend='both')

# add color bar
labels = [str(item) for item in levels]
cax,kw = matplotlib.colorbar.make_axes(ax2,ticks=levels,location='right',pad=0.03,shrink=0.7)
cbar = fig.colorbar(cs,cax=cax,**kw)
cbar.ax.set_yticklabels(labels)
cbar.set_label('Precipitation $[mm/day]$',size=10)

# crop extent of map to lat/lon of data
ax2.set_extent(az_cood)

# add lat lon gridline
gl = ax2.gridlines(draw_labels=True,
             xlocs=np.arange(-180, 180, 1.),
             ylocs=np.arange(-90, 90, 1.),
             linewidth=0.5, color='k', alpha=0.5, linestyle=':')
gl.xlabels_top = None
gl.ylabels_right = None
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# set plot title
ax2.set_title('2010-07-10 :: Stage IV Observations', loc='center',pad=5,fontsize=12) 

# add map features to plot
#ax2.add_feature(cartopy.feature.LAND)
#ax2.add_feature(cartopy.feature.OCEAN)
ax2.add_feature(cartopy.feature.COASTLINE)
ax2.add_feature(cartopy.feature.STATES,linewidth=0.5)
ax2.add_feature(cartopy.feature.BORDERS, linestyle=':',linewidth=0.5)

# save plot to same directory as script
plotfile = 'model_vs_obs_precip_plot.png'
sf = fig.savefig(plotfile, dpi=300, bbox_inches='tight')
plt.show()